# Loading Chicago Data

Creating a connection the gridMET planetary dataset

In [1]:
import pystac_client
import planetary_computer

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

asset = catalog.get_collection("gridmet").assets["zarr-abfs"]
asset

<Asset href=abfs://gridmet/gridmet.zarr>

Using xarray to read it:

In [2]:
import xarray as xr

ds = xr.open_zarr(
    asset.href,
    storage_options=asset.extra_fields["xarray:storage_options"],
    **asset.extra_fields["xarray:open_kwargs"]
)
ds

<xarray.Dataset>
Dimensions:                                    (time: 15341, lat: 585,
                                                lon: 1386, crs: 1)
Coordinates:
  * crs                                        (crs) uint16 3
  * lat                                        (lat) float64 49.4 ... 25.07
  * lon                                        (lon) float64 -124.8 ... -67.06
  * time                                       (time) datetime64[ns] 1979-01-...
Data variables:
    air_temperature                            (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    burning_index_g                            (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    dead_fuel_moisture_1000hr                  (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    dead_fuel_moisture_100hr                   (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    mean_vapor_pressure_deficit                (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    potential_evapotranspiration               (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    precipitation_amount                       (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    relative_humidity                          (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    specific_humidity                          (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    surface_downwelling_shortwave_flux_in_air  (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    wind_from_direction                        (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    wind_speed                                 (time, lat, lon) float32 dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                CF-1.6
    author:                     John Abatzoglou - University of Idaho, jabatz...
    coordinate_system:          EPSG:4326
    date:                       02 July 2019
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_bounds_crs:      EPSG:4326
    ...                         ...
    geospatial_lon_units:       decimal_degrees east
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

Creating the Chicago polygon and time period:

In [3]:
# Define the bounding box for Chicago (as discussed)
chicago_lat_min = 41.5781
chicago_lat_max = 42.1781
chicago_lon_min = -87.9298
chicago_lon_max = -87.3298

# Slice the dataset for the date range 2017-2020 and the Chicago region
ds_chicago = ds.sel(
    time=slice("2017-01-01", "2020-12-31"),
    lat=slice(chicago_lat_max, chicago_lat_min),  # Note: latitudes are descending
    lon=slice(chicago_lon_min, chicago_lon_max)
)

# Preview the selection
ds_chicago


<xarray.Dataset>
Dimensions:                                    (time: 1461, lat: 14, lon: 14,
                                                crs: 1)
Coordinates:
  * crs                                        (crs) uint16 3
  * lat                                        (lat) float64 42.15 ... 41.61
  * lon                                        (lon) float64 -87.89 ... -87.35
  * time                                       (time) datetime64[ns] 2017-01-...
Data variables:
    air_temperature                            (time, lat, lon) float32 dask.array<chunksize=(10, 14, 14), meta=np.ndarray>
    burning_index_g                            (time, lat, lon) float32 dask.array<chunksize=(10, 14, 14), meta=np.ndarray>
    dead_fuel_moisture_1000hr                  (time, lat, lon) float32 dask.array<chunksize=(10, 14, 14), meta=np.ndarray>
    dead_fuel_moisture_100hr                   (time, lat, lon) float32 dask.array<chunksize=(10, 14, 14), meta=np.ndarray>
    mean_vapor_pressure_deficit                (time, lat, lon) float32 dask.array<chunksize=(10, 14, 14), meta=np.ndarray>
    potential_evapotranspiration               (time, lat, lon) float32 dask.array<chunksize=(10, 14, 14), meta=np.ndarray>
    precipitation_amount                       (time, lat, lon) float32 dask.array<chunksize=(10, 14, 14), meta=np.ndarray>
    relative_humidity                          (time, lat, lon) float32 dask.array<chunksize=(10, 14, 14), meta=np.ndarray>
    specific_humidity                          (time, lat, lon) float32 dask.array<chunksize=(10, 14, 14), meta=np.ndarray>
    surface_downwelling_shortwave_flux_in_air  (time, lat, lon) float32 dask.array<chunksize=(10, 14, 14), meta=np.ndarray>
    wind_from_direction                        (time, lat, lon) float32 dask.array<chunksize=(10, 14, 14), meta=np.ndarray>
    wind_speed                                 (time, lat, lon) float32 dask.array<chunksize=(10, 14, 14), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                CF-1.6
    author:                     John Abatzoglou - University of Idaho, jabatz...
    coordinate_system:          EPSG:4326
    date:                       02 July 2019
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_bounds_crs:      EPSG:4326
    ...                         ...
    geospatial_lon_units:       decimal_degrees east
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

In [4]:
# Estimate memory usage of the subset
total_bytes = sum(v.data.nbytes for v in ds_chicago.data_vars.values())
print(f"Estimated memory size for chicago_data: {total_bytes / 1e9:.2f} GB")


Estimated memory size for chicago_data: 0.01 GB


## Weekly resampling each pair of (lat,long)

In [5]:
agg_methods = {
    "air_temperature": "mean",  # Average temperature per week
    "burning_index_g": "max",   # Use max for fire danger indicators
    "dead_fuel_moisture_1000hr": "mean",  # Slow-changing moisture → mean
    "dead_fuel_moisture_100hr": "mean",   # Same as above
    "mean_vapor_pressure_deficit": "mean",  # Continuous variable → mean
    "potential_evapotranspiration": "sum",  # Accumulated over time
    "precipitation_amount": "sum",  # Rainfall is cumulative
    "relative_humidity": "mean",  # Weekly average humidity
    "specific_humidity": "mean",  # Same idea
    "surface_downwelling_shortwave_flux_in_air": "mean",  # Radiative flux → mean
    "wind_from_direction": "mean",  # Can use circular stats, but mean works as approx
    "wind_speed": "mean"  # Weekly average wind speed
}

In [6]:
# Step 1: Extract only relevant variables
ds_subset = ds_chicago[list(agg_methods.keys())]

# Step 2: Create a new Dataset by applying appropriate method
ds_weekly = xr.Dataset({
    var: getattr(ds_subset[var].resample(time="1W"), method)()
    for var, method in agg_methods.items()
})

# Step 3 (optional): Inspect result
print(ds_weekly)

<xarray.Dataset>
Dimensions:                                    (lat: 14, lon: 14, time: 210)
Coordinates:
  * lat                                        (lat) float64 42.15 ... 41.61
  * lon                                        (lon) float64 -87.89 ... -87.35
  * time                                       (time) datetime64[ns] 2017-01-...
Data variables:
    air_temperature                            (time, lat, lon) float32 dask.array<chunksize=(1, 14, 14), meta=np.ndarray>
    burning_index_g                            (time, lat, lon) float32 dask.array<chunksize=(1, 14, 14), meta=np.ndarray>
    dead_fuel_moisture_1000hr                  (time, lat, lon) float32 dask.array<chunksize=(1, 14, 14), meta=np.ndarray>
    dead_fuel_moisture_100hr                   (time, lat, lon) float32 dask.array<chunksize=(1, 14, 14), meta=np.ndarray>
    mean_vapor_pressure_deficit                (time, lat, lon) float32 dask.array<chunksize=(1, 14, 14), meta=np.ndarray>
    potential_evapotransp

If we were to do this:
```python
df_weekly = ds_weekly.to_dataframe().reset_index()
``` 
gives: One row per week per lat-lon pair.

So if you have 14 latitudes × 14 longitudes = 196 grid points, you’ll get:

`210 weeks × 196 = 41,160 rows`

Each row corresponds to the climate values at a specific grid cell in Chicago for a specific week.

If we aggregate it for the **whole of Chicago:**

In [7]:
ds_weekly_avg = ds_weekly.mean(dim=["lat", "lon"])

## DataFraming it

Let's convert it into an actual DataFrame:

In [8]:
df_weekly_avg = ds_weekly_avg.to_dataframe().reset_index()

c:\Users\lucas\anaconda3\Lib\site-packages\dask\array\reductions.py:654: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)


In [10]:
df_weekly_avg

,time,air_temperature,burning_index_g,dead_fuel_moisture_1000hr,dead_fuel_moisture_100hr,mean_vapor_pressure_deficit,potential_evapotranspiration,precipitation_amount,relative_humidity,specific_humidity,surface_downwelling_shortwave_flux_in_air,wind_from_direction,wind_speed
0,2017-01-01,267.272949,20.210526,20.237595,17.388723,0.231128,0.521939,0.000000,47.122559,0.002402,85.250381,149.503754,2.747369
1,2017-01-08,259.829437,37.323307,19.652416,15.695059,0.101031,2.853061,3.989796,51.845432,0.001655,61.916435,240.713226,5.771750
2,2017-01-15,267.168518,37.759399,19.084854,16.336735,0.211171,5.117857,14.477041,51.307091,0.002353,63.396881,212.327591,5.535016
3,2017-01-22,274.063629,24.977444,20.365627,18.864016,0.156541,3.608674,33.237755,72.376579,0.004253,67.344368,172.853912,3.732223
4,2017-01-29,270.624817,29.353384,19.981312,17.215576,0.129957,3.586225,1.519388,71.084427,0.003011,85.066605,266.027924,5.168529
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,2020-12-06,271.148651,38.924812,16.582600,13.515144,0.282685,8.919388,0.026531,50.138565,0.002563,80.122559,209.609039,4.581311
206,2020-12-13,272.913727,32.112782,16.585823,16.698496,0.239517,6.604592,29.976019,60.122444,0.003523,69.939949,212.665939,3.796348
207,2020-12-20,269.970245,31.909775,17.288507,15.826426,0.212170,6.932143,0.012755,54.240707,0.002500,71.791298,193.200867,4.119120
208,2020-12-27,266.728271,42.037594,16.501934,14.083137,0.255081,9.822450,2.012245,42.903866,0.002365,72.948555,235.408173,6.623738


In [11]:
# Step 3: Save to disk
df_weekly_avg.to_csv("data/chicago_weekly_avg.csv", index=False)
df_weekly_avg.to_parquet("data/chicago_weekly_avg.parquet", index=False)

print("✅ Data saved as CSV and Parquet!")

✅ Data saved as CSV and Parquet!
